# Data Filtering with SQLAlchemy

In [9]:
import sqlalchemy as db
import pandas as pd
%run auto_sql.py

In [2]:
# path to sqlite file
file = '/home/ilves/Documents/jupyter_anaconda/learning_new/sql_stuff/census.sqlite'

# creating engine with sqlite dialect
engine = db.create_engine('sqlite:///' + file)

In [3]:
# printing table names without actually connecting
print(engine.table_names())

['census', 'state_fact']


In [7]:
# Print full table metadata
print(repr(metadata.tables['census']))

Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


In [4]:
# create metadata and table objects
metadata = db.MetaData()
census = db.Table('census', metadata, autoload=True, autoload_with=engine)

# where

In [5]:
column_names('census', engine)

['state', 'sex', 'age', 'pop2000', 'pop2008']


In [10]:
pd.read_sql('select * from census', engine).head(3)

,state,sex,age,pop2000,pop2008
0,Illinois,M,0,89600,95012
1,Illinois,M,1,88445,91829
2,Illinois,M,2,88729,89547


**SQL** : <br>
SELECT * FROM census <br>
WHERE sex = F <br>

### SQLAlchemy

In [14]:
# writing down the statement in sqlalchemy way
query = db.select([census]).where(census.columns.sex == 'F')

# creating connection
connection = engine.connect()

results = connection.execute(query).fetchall()

for res in results[:3]:
    print(res.state, res.age, res.pop2000)

Illinois 0 85910
Illinois 1 84396
Illinois 2 84764


### pandas

In [15]:
pd.read_sql_query("select * from census where sex == 'F'", engine).head(3)

,state,sex,age,pop2000,pop2008
0,Illinois,F,0,85910,90286
1,Illinois,F,1,84396,88126
2,Illinois,F,2,84764,86291


# in

**SQL** : <br>
SELECT state, sex<br>
FROM census<br>
WHERE state IN (Texas, New York)<br>

## SQLAlchemy

In [16]:
query = db.select([census.columns.state, census.columns.sex]).where(census.columns.state.in_(['Texas', 'New York']))

results = connection.execute(query).fetchall()

for res in results[:3]:
    print(res)

('New York', 'M')
('New York', 'M')
('New York', 'M')


## pandas

In [19]:
pd.read_sql_query("select state, sex from census where state in ('Texas', 'New York')", engine).head(3)

,state,sex
0,New York,M
1,New York,M
2,New York,M


## and, or, not

**SQL** :<br>
SELECT * FROM census <br>
WHERE state = 'California' AND NOT sex = 'M'<br>

In [20]:
pd.read_sql_query("SELECT * FROM census WHERE state = 'California' AND NOT sex = 'M'", engine).head(3)

,state,sex,age,pop2000,pop2008
0,California,F,0,239605,274356
1,California,F,1,236543,269140
2,California,F,2,240010,262556
